In [2]:
!nvidia-smi

Thu Mar 16 00:56:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    56W / 300W |  31374MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [4]:
import sys
import os
import shutil
sys.path.append("../../")
sys.path.append("../")

In [5]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory
from datasets import DataManager

ImportError: cannot import name 'DataManager' from 'datasets' (/opt/conda/envs/ml/lib/python3.10/site-packages/datasets/__init__.py)

In [ ]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models.mobilenet_v2 import get_mobilenet

In [6]:
data_root = './'

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


In [8]:
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(224, scale=(crop_scale, 1.0)),
    transforms.RandomHorizontalFlip()])
val_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize(256),
                                transforms.CenterCrop(224)
                            ])
test_transform = val_transform

In [9]:
class CFG:
    def __init__(self):
        self.batch_size = 64
        self.valid_size = 0.1
        self.num_train = 0
        self.epochs = 256
        self.optimizer = torch.optim.Adam
        self.lr = 0.001
        self.momentum = 0.9
        self.weight_decay = 0
        self.save_path = './save_path_1024_mbnetv2_cifar100_4_FP'
        self.data_path = ''
        self.label_smooth = 0.1
        self.workers = 4
        self.device = 'cuda'
        self.dataset = 'c100'  #valid options are c100, c10 and tin
        self.num_classes = 100 #10 for CIFAR10 // 100 for CIFAR100 // 200 for Tiny ImageNet 
        self.num_fp = 2

In [10]:
args = CFG()
args = args.__dict__
data_object = DataManager(args)
trainloader, valloader, testloader = data_object.prepare_data(train_transform, val_transform)
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


In [11]:
model = get_mobilenet(num_classes=args['num_classes'], num_fp=args['num_fp'])

In [12]:
algo = BirealNet(model, dataloaders, **args)

In [ ]:
algo.binarize()

01/15 01:47:39 PM CFG = {'batch_size': 64, 'valid_size': 0.1, 'num_train': 0, 'epochs': 256, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'momentum': 0.9, 'weight_decay': 0, 'save_path': './save_path_1024_mbnetv2_cifar100_4_FP', 'data_path': '', 'label_smooth': 0.1, 'workers': 4, 'device': 'cuda', 'dataset': 'c100', 'num_classes': 100, 'num_fp': 2}
01/15 01:47:39 PM MobileNetv2(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [0][  0/704]	Time  2.695 ( 2.695)	Data  0.955 ( 0.955)	Loss 4.6044e+00 (4.6044e+00)	Acc@1   0.00 (  0.00)	Acc@5   3.12 (  3.12)
Epoch: [0][  1/704]	Time  0.349 ( 1.522)	Data  0.001 ( 0.478)	Loss 4.7215e+00 (4.6630e+00)	Acc@1   0.00 (  0.00)	Acc@5   4.69 (  3.91)
Epoch: [0][  2/704]	Time  0.352 ( 1.132)	Data  0.001 ( 0.319)	Loss 4.8596e+00 (4.7285e+00)	Acc@1   1.56 (  0.52)	Acc@5   1.56 (  3.12)
Epoch: [0][  3/704]	Time  0.354 ( 0.938)	Data  0.001 ( 0.239)	Loss 4.8832e+00 (4.7672e+00)	Acc@1   0.00 (  0.39)	Acc@5   4.69 (  3.52)
Epoch: [0][  4/704]	Time  0.350 ( 0.820)	Data  0.001 ( 0.192)	Loss 4.8639e+00 (4.7865e+00)	Acc@1   0.00 (  0.31)	Acc@5   1.56 (  3.12)
Epoch: [0][  5/704]	Time  0.354 ( 0.742)	Data  0.001 ( 0.160)	Loss 5.0803e+00 (4.8355e+00)	Acc@1   1.56 (  0.52)	Acc@5   6.25 (  3.65)
Epoch: [0][  6/704]	Time  0.356 ( 0.687)	Data  0.001 ( 0.137)	Loss 5.0310e+00 (4.8634e+00)	Acc@1   1.56 (  0.67)	Acc@5   4.69 (  3.79)
Epoch: [0][  7/704]	Time  0.351 ( 0.645)	Data  0.001 ( 

In [10]:
chk=torch.load('./save_path_1024_mbnetv2_cifar100_2fp/c100-model_best.pth.tar')

In [11]:
chk['best_top1_acc']

tensor(66.2200)